# IMPORTING LIBRARIES

In [107]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



# IMPORTING DATA AND CONVERTING IT INTO DATAFRAME

In [108]:
data= np.load('galaxy_catalogue.npy')
df = pd.DataFrame(data)


# Shuffeling dataframe for unbiasing data

In [109]:

from sklearn.utils import shuffle
df = shuffle(df)
df.reset_index(inplace=False, drop=True)


,u-g,g-r,r-i,i-z,ecc,m4_u,m4_g,m4_r,m4_i,m4_z,petroR50_u,petroR50_r,petroR50_z,petroR90_u,petroR90_r,petroR90_z,class
0,1.83027,0.84431,0.39433,0.30880,0.964706,2.475909,2.442293,2.442839,2.434965,2.419468,9.043689,5.467977,5.004704,22.523330,17.665140,17.140940,elliptical
1,2.00159,0.89582,0.47336,0.25935,0.853795,2.076495,2.455420,2.430792,2.462987,2.438677,3.971475,3.842477,3.323616,11.301030,9.285395,8.744328,merger
2,1.79945,0.79592,0.40726,0.27453,0.510135,2.415254,2.452152,2.443151,2.440272,2.383487,7.026027,5.447197,4.933511,21.875780,16.251430,15.299280,merger
3,1.92302,0.92581,0.40007,0.34945,0.775080,2.451854,2.376309,2.391045,2.414250,2.377393,3.273369,3.025038,2.891787,11.252560,9.868087,9.006919,elliptical
4,1.91042,0.87266,0.43484,0.29740,0.794391,2.343216,2.326988,2.362205,2.372770,2.385539,13.680320,4.188968,3.460898,27.653440,19.199850,16.082200,merger
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,1.33324,0.70361,0.36660,0.26710,0.666506,2.036027,2.106161,2.284671,2.339318,2.361116,4.880836,4.103659,3.748782,9.431236,8.418779,7.793340,spiral
776,1.92181,0.91974,0.41321,0.31200,0.725056,2.474488,2.440215,2.442940,2.441271,2.445905,4.279113,4.174338,4.066437,12.210550,13.208710,13.379900,elliptical
777,1.86894,0.89636,0.40368,0.26953,0.493673,2.409214,2.479733,2.467336,2.488909,2.481156,7.425199,4.275969,3.420454,16.354200,11.877800,9.277365,merger
778,1.92493,0.98640,0.41899,0.31943,0.778894,2.303315,2.373983,2.379314,2.396000,2.388731,5.046247,2.989523,2.694031,12.252950,9.094408,8.739716,elliptical


# PREPARING Y FOR INDIVIDUAL CLASSIFICATION

In [110]:
X = df.iloc[:,:10]

y=pd.DataFrame(df.iloc[:,-1])
y.shape
y_1 = np.where(y=="merger",1,0)    

y_1.reshape((780,1))

y_2 = np.where(y=="elliptical",1,0)

y_3 = np.where(y=="spiral",1,0)


# ADDITIONAL FEATURES

In [111]:
X['conc_u']=(df['petroR50_u'])/(df['petroR90_u'])
X['conc_r']=df['petroR50_r']/df['petroR90_r']
X['conc_z']= df['petroR50_z']/df['petroR90_z']

# CREATING Std SCALAR VECTOR 

In [112]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

# SPLITTING TEST AND TRAIN DATASET

In [113]:
m = 600
f = 13
X_train = X[:m,:f]
X_train = np.c_[np.ones(len(X_train),dtype='int64'),X_train]
y_train_1 = y_1[:m]
y_train_2 = y_2[:m]
y_train_3 = y_3[:m]
X_test = X[m:,:f]
X_test = np.c_[np.ones(len(X_test),dtype='int64'),X_test]
y_test = y[m:]


# ENCODING Y BEACUSE INITIALLY IT WAS STRING

In [114]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
y_encoded = ordinal_encoder.fit_transform(y)
y_encoded = y_encoded[:600,0]

y_encoded_test = y_encoded[600:]
y_encoded = y_encoded.reshape(600,1)
y_encoded_test = ordinal_encoder.fit_transform(y_test)

# INITIATE COFFICIENT

In [115]:
# Initial Coefficients
theta = np.zeros((14,1))
alpha = 0.005
loops = 2000


# DEFINITION OF SIGMOID FUNCTION

In [116]:
def sigmoid(theta, x):
    z = np.dot(x,theta)
    return(1/(1+np.exp(-z)))

# DEFINITION OF COST FUNCTION

In [117]:
h= sigmoid(theta,X_train)

In [118]:
def cost(h,y):
    m=len(y)
    return ((1/m)*np.sum((-y.T*np.log(h)-(1-y).T*np.log(1-h)),axis = 1, keepdims = True))


# GRADIENT DESCENT

In [119]:
c= cost(h,y_train_1)

In [120]:
#import numpy.matlib
def gradient_descent(x,y,theta,alpha):
    m = len(y)
    #for i in range(0,2000):
    g= sigmoid(theta,x)
        #g1= g-y
        #G=np.matlib.repmat(g-y,1,14)
        #G.shape
    theta = theta - (alpha/m)*(np.dot(x.T, (g-y)))
    return theta


# FITTING GRADIENT DESCENT

In [121]:
def fit(x,y,loops,alpha):
    theta = np.zeros((14,1))
    for loop in range(loops):
        theta=gradient_descent(x,y,theta,alpha)
    return theta

# COMPUTATION OF THETA_MIN FOR INDIVIDUAL CLASSES AND CREATION OF ALL_THETA WHICH CONTAINS THETA_MIN FOR ALL THE LABELS

In [122]:
theta_min_1 = fit(X_train,y_train_1,loops,alpha)
theta_min_2 = fit(X_train,y_train_2,loops,alpha)
theta_min_3 = fit(X_train,y_train_3,loops,alpha)

all_theta = np.zeros((3, 14))
for i in range(0,14):
    all_theta[0,i] = theta_min_2.T[:,i]
    all_theta[1,i] = theta_min_1.T[:,i]
    all_theta[2,i] = theta_min_3.T[:,i]


In [123]:
h_theta=sigmoid(all_theta.T,X_train)
h_theta.shape

p = np.amax(h_theta, axis=1,keepdims = True)  #Function amax used to get the maximum value of probability row wise
result = np.where(h_theta == p) #getting the index of maximum values as probability i.e predicted y.

probability = result[1]
probability = probability.reshape(600,1)




In [124]:
h_theta_test = sigmoid(all_theta.T,X_test)
h_theta_test.shape
p = np.amax(h_theta_test, axis=1,keepdims = True)
result_test = np.where(h_theta_test == p)

probability_test = result_test[1]
probability_test = probability_test.reshape(180,1)



In [125]:
#import matplotlib.pyplot as plt
#plt.hist(Cost,bins=50)
predict = (probability == y_encoded).reshape((600,1))  #checks if our probability is eual to the value of y for given data
pe=predict.astype(int) #converting into int (binary values)
predict_test = (probability_test == y_encoded_test).reshape((180,1)) 
pe_test = predict_test.astype(int)
#print(pe_test)
print("Accuracy train : ",np.mean(pe) * 100)        #mean * 100 of pe will be accuracy
print("Accuracy test : ",np.mean(pe_test)*100)

Accuracy train :  81.16666666666667
Accuracy test :  77.22222222222223
